In [7]:
# 为了兼容python2和python3
from __future__ import division,print_function,unicode_literals

# 导入相应的包
import numpy as np
import pandas as pd
import os

# 为了保持稳定性一致
np.random.seed(42)

# 画图设置
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

import warnings 
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# mpl.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
sns.set_style('darkgrid')                      # darkgrid,whitegrid,dark,white,ticks

In [8]:
import tensorflow as tf
print(tf.__version__)

1.8.0


# 1 创造第一个图谱，然后运行它

In [9]:
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [10]:
# 手动初始化
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

print(result)

42


In [11]:
# 使用global_variables_initializer() 函数初始化
# 它实际上没有立即执行初始化，而是在图谱中创建一个当程序运行时所有变量都会初始化的节点
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42


In [12]:
# jupyter和python shell中创建交互式  
# init = tf.global_variables_initializer()
# sess = tf.InteractiveSession()  
# init.run()  
# result = f.eval()  
# print(result)  
# sess.close()  

# 2 管理图谱graph

In [13]:
# 默认graph
reset_graph()
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [14]:
# 创建独立的graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [15]:
x2.graph is tf.get_default_graph()

False

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


In [17]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


# 3 逻辑回归LR

In [18]:
# Use Tensorflow
import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
#np.c_按colunm来组合array 
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
print(theta_value) 

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [19]:
# Compare with pure NumPy
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [20]:
# Compare with Scikit-Learn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


# 4 批量梯度下降Batch Gradient Descent

In [21]:
# 梯度下降需要首先缩放特征向量
# 我们可以使用TF来做到这一点，但现在让我们使用Scikit-Learn。
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

print(scaled_housing_data_plus_bias.mean(axis=0))
print("===================================================================")
print(scaled_housing_data_plus_bias.mean(axis=1))
print("===================================================================")
print(scaled_housing_data_plus_bias.mean())
print("===================================================================")
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


# 5 手动计算渐变

In [22]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.5396291
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [23]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614529 ],
       [-0.6375279 ]], dtype=float32)

# 6 Using autodiﬀ——使用自动求微分

TensorFlow 的自动计算梯度功能可以计算这个公式：它可以自动高效地为您计算梯度。 只需用以下面这行代码替换上一节中代码的gradients = ...行，代码将继续工作正常

In [24]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate * gradients)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.71450037
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
Best theta:
[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178365]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


# 7 使用优化器

In [25]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
Best theta:
[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178365]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


# 使用动量优化器

In [26]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Best theta:
[[ 2.068558  ]
 [ 0.82962847]
 [ 0.11875335]
 [-0.26554456]
 [ 0.3057109 ]
 [-0.00450249]
 [-0.03932662]
 [-0.8998645 ]
 [-0.8705207 ]]


# 8 将数据提供给训练算法

In [27]:
# Placeholder nodes
reset_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print("++++++++++++++")
print(B_val_2)

[[6. 7. 8.]]
++++++++++++++
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [28]:
# Mini-batch Gradient Descent
reset_graph()
import numpy as np  
from sklearn.datasets import fetch_california_housing  
import tensorflow as tf  
from sklearn.preprocessing import StandardScaler  
  
housing = fetch_california_housing()  
m, n = housing.data.shape  
print("数据集:{}行,{}列".format(m,n))  
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]  
scaler = StandardScaler()  
scaled_housing_data = scaler.fit_transform(housing.data)  
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]  
  
n_epochs = 1000  
learning_rate = 0.01  
  
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")  
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")  
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")  
y_pred = tf.matmul(X, theta, name="predictions")  
error = y_pred - y  
mse = tf.reduce_mean(tf.square(error), name="mse")  
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)  
training_op = optimizer.minimize(mse)  
  
init = tf.global_variables_initializer()  
  
n_epochs = 10  
batch_size = 100  
n_batches = int(np.ceil(m / batch_size)) # ceil() 方法返回 x 的值上限 - 不小于 x 的最小整数。  
  
def fetch_batch(epoch, batch_index, batch_size):  
    know = np.random.seed(epoch * n_batches + batch_index)
    print("我是know:",know)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices]  
    y_batch = housing.target.reshape(-1, 1)[indices]  
    return X_batch, y_batch  
  
with tf.Session() as sess:  
    sess.run(init)  
  
    for epoch in range(n_epochs):  
        for batch_index in range(n_batches):  
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)  
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  
  
    best_theta = theta.eval()  
  
print(best_theta)  

数据集:20640行,8列
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: Non

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255152]
 [-0.31211877]
 [ 0.38510376]
 [ 0.00434168]
 [-0.0123295 ]
 [-0.83376896]
 [-0.8030471 ]]


# 9 保存和恢复模型

In [29]:
reset_graph()

n_epochs = 1000                                                                       
learning_rate = 0.01                                                                  

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      
error = y_pred - y                                                                    
mse = tf.reduce_mean(tf.square(error), name="mse")                                    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            
training_op = optimizer.minimize(mse)                                                 

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                
            save_path = saver.save(sess, "./save_model/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./save_model/my_model_final.ckpt")

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [30]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.3477088 ],
       [ 0.36178365],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145283],
       [-0.6375278 ]], dtype=float32)

In [31]:
# 恢复模型
with tf.Session() as sess:
    saver.restore(sess, "./save_model/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./save_model/my_model_final.ckpt


In [32]:
np.allclose(best_theta, best_theta_restored)

True

# 10 Visualizing the graph

In [33]:
# from tensorflow_graph_in_jupyter import show_graph
# show_graph(tf.get_default_graph())

In [34]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # not shown
    
file_writer.close()
best_theta

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None

我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None
我是know: None


array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255152],
       [-0.31211877],
       [ 0.38510376],
       [ 0.00434168],
       [-0.0123295 ],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

In [35]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape
print("数据集:{}行,{}列".format(m,n))
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
file_writer.close()
print(best_theta)

数据集:20640行,8列
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255152]
 [-0.31211877]
 [ 0.38510376]
 [ 0.00434168]
 [-0.0123295 ]
 [-0.83376896]
 [-0.8030471 ]]
